<h1 style="text-align: center;"> LOCUS COERULEUS (LC) - MEAN INTENSITY EXTRACTION </h1>

The following code was used to extract mean signal intensities for the LC (left, right, and both hemispheres). 
- Initially, the intensities from the SNpc, LC, putamen, and WM were only saved after all files for all participants had been processed. Should the Linux server where extraction was performed crash, all processed data would be lost, resulting in a significant loss of time. 
- When the extraction for the LC crashed, the code was altered to save mean intensities for each file immediately after processing. 
- This change was applied only to the LC extraction, as it took longer than the extraction processes for other regions such as the SN, WM, and putamen.

In [ ]:
import os
import pandas as pd
import subprocess
import nibabel as nib

In [ ]:
#define base directory and LC mask names
base_dir = "/hus/home/oliwin/data/study_name"

LC_mask = f"{base_dir}/LC_mask.nii.gz"
mask_lc_l = f"{base_dir}/LC_L_mask.nii.gz"
mask_lc_r = f"{base_dir}/LC_R_mask.nii.gz"

mask_img = nib.load(LC_mask)
mask_data = mask_img.get_fdata()

# get dimensions and split along x-axis
x_mid = mask_data.shape[0] // 2

# create left mask
left_mask = mask_data.copy()
left_mask[x_mid:] = 0

# create right mask
right_mask = mask_data.copy()
right_mask[:x_mid] = 0

# save the left and right masks
left_mask_img = nib.Nifti1Image(left_mask, affine=mask_img.affine)
right_mask_img = nib.Nifti1Image(right_mask, affine=mask_img.affine)

nib.save(left_mask_img, mask_lc_l)
nib.save(right_mask_img, mask_lc_r)

print(f"Left mask saved at: {mask_lc_l}")
print(f"Right mask saved at: {mask_lc_r}")

Left mask saved at: /hus/home/oliwin/data/study_name/LC_L_mask.nii.gz
Right mask saved at: /hus/home/oliwin/data/study_name/LC_R_mask.nii.gz


In [ ]:
# dictionary to store participant-wise data
patients_data = {}

# loop through subdirectories 'FSL' in base_dir (recursively)
for root, dirs, files in os.walk(base_dir):
    if "FSL" in dirs:
        fsl_dir = os.path.join(root, "FSL")
        
        # extract participant number from folder name
        patient_number = os.path.basename(root)
        
        # loop through NIfTI files in the FSL directory
        for filename in os.listdir(fsl_dir):
            # process only files with "mni", "nii.gz"
            if (
                "mni" in filename
                and "mask" not in filename
                and "new" not in filename
                and "flirt" not in filename
                and "flirt_masked" not in filename
                and not filename.endswith(".mat")
                and filename.endswith(".nii.gz")
            ):
                input_file = os.path.join(fsl_dir, filename)

                
                print(f"Processing file: {input_file}")

                # loop through the LC masks (left, right and combined) - each MRI file is processed three times
                for mask_file in [LC_mask, mask_lc_r, mask_lc_l]:
                    try:
                        # use FLIRT to perform the registration of the LC masks
                        flirt_output = f"{input_file}_flirt_{os.path.basename(mask_file).split('.')[0]}"
                        flirt_matrix = f"{flirt_output}.mat"
                        flirt_command = f"flirt -in {input_file} -ref {mask_file} -out {flirt_output} -omat {flirt_matrix}"
                        process_flirt = subprocess.run(flirt_command, shell=True, capture_output=True, text=True)

                        if process_flirt.returncode != 0:
                            print(f"Error applying FLIRT to {filename} with mask {mask_file}: {process_flirt.stderr}")
                            continue  # Continue with the next mask for same file

                        # use fslmaths to apply the masks and threshold negative values to zero
                        flirt_masked = f"{flirt_output}_masked"
                        fslmaths_command = f"fslmaths {flirt_output} -mas {mask_file} -thr 0 {flirt_masked}"
                        process_fslmaths = subprocess.run(fslmaths_command, shell=True, capture_output=True, text=True)

                        if process_fslmaths.returncode != 0:
                            print(f"Error applying fslmaths to {filename} with mask {mask_file}: {process_fslmaths.stderr}")
                            continue  # Continue with the next mask for same file

                        # use fslstats to extract mean intensity within the region
                        stats_command = f"fslstats {flirt_masked} -M"  # -M for Mean Intensity 
                        process_stats = subprocess.run(stats_command, shell=True, capture_output=True, text=True)

                        if process_stats.returncode != 0:
                            print(f"Error processing {filename}: {process_stats.stderr}")
                            continue  # Continue with the next mask for same file

                        
                        stats = process_stats.stdout.strip().split()
                        mean_intensity = float(stats[0]) 

                        # remove intermediate files and temporary NIfTI outputs
                        try:
                            for file_to_delete in [flirt_matrix, flirt_output + ".nii.gz", flirt_masked + ".nii.gz"]:
                                if os.path.exists(file_to_delete):
                                    os.remove(file_to_delete)
                                else:
                                    print(f"File not found: {file_to_delete}")
                        except OSError as e:
                            print(f"Error deleting files for {filename}: {e}")

                        # add data to the participant's entry
                        if patient_number not in patients_data:
                            patients_data[patient_number] = {}

                        mask_name = os.path.basename(mask_file).split('.')[0]  
                        filename_base = filename.replace('.nii.gz', '')  
                        mask_key = f'{mask_name}_{filename_base}'

                        patients_data[patient_number][mask_key] = mean_intensity

                    except Exception as e:
                        print(f"Error processing {filename} with mask {mask_file} for patient {patient_number}: {e}")
                        continue  # continue to the next mask for same file

# preparing the dataframe to store mean intensities
header = ['Patient Number']
all_keys = set()
for intensities in patients_data.values():
    all_keys.update(intensities.keys())
header.extend(sorted(all_keys)) 

output_data = []
for patient_number, intensities in patients_data.items():
    row = [patient_number]
    for col in header[1:]:
        row.append(intensities.get(col, None))  # use None for missing values
    output_data.append(row)

# creating dataframe
output_df = pd.DataFrame(output_data, columns=header)

# saving the dataframe as csv for further analysis
output_file = 'mri_data_LC.csv'  
output_df.to_csv(output_file, index=False)
print(f"Transformed CSV saved to {output_file}")

Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_HPF_filled_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_filled_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_filled_MIP_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_mpSWIM_ECHO-3_e3_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_pSWIM_ECHO-3_e3_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simCSF_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simFLAIR_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simGM_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simWM_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_sim_GRE_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/st

Traceback (most recent call last):
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/tmp/ipykernel_839480/323384633.py", line 106, in <module>
    output_df.to_csv(output_file, index=False)
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/pandas/util/_decorators.py", line 333, in wrapper
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/pandas/core/generic.py", line 3967, in to_csv
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/pandas/io/formats/format.py", line 987, in to_csv
ModuleNotFoundError: No module named 'pandas.io.formats.csvs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exc

In [ ]:
print(output_df)

  Patient Number  LC_L_mask_STAGE_CROWN_PD_MAP_reg_mni  \
0       0000D71C                            769.477901   
1       0000D2D0                            665.604972   
2       0000997A                            614.980663   
3       00008904                            570.397790   
4       00008289                            592.055249   
5       00005316                            713.991713   
6       00002530                            565.196133   
7       0000437C                            760.701657   
8       000096EB                            821.886740   
9       000077F0                            604.060773   

   LC_L_mask_STAGE_CROWN_R2S_A2_reg_mni  LC_L_mask_STAGE_CROWN_R2S_reg_mni  \
0                           2104.563536                        1956.165746   
1                           2072.629834                        3735.759669   
2                              0.000000                        1763.124309   
3                           1407.839779          

In [ ]:
# saving the mean intensities extracted before server crash to csv
output_file = 'mri_data_LC.csv'  
output_df.to_csv(output_file, index=False)
print(f"Transformed CSV saved to {output_file}")

Transformed CSV saved to mri_data_LC.csv


In [ ]:
# loading the CSV file from first extraction that crashed and cross referencing with WM CSV file to extract 
# patient-id for participants that were missed
lc_file = "mri_data_LC.csv"
white_file = "mri_data_white.csv" 

lc_df = pd.read_csv(lc_file)
white_df = pd.read_csv(white_file)

# extract 'Patient Number' column from both files
lc_patients = set(lc_df['Patient Number'])
white_patients = set(white_df['Patient Number'])

# identifying participants in wm csv but not in LC csv
missing_patients = white_patients - lc_patients

print("Missing patients:")
for patient in sorted(missing_patients):
    print(patient)

Missing patients:
00000172
000003B4
00000A67
00000EAE
00001573
00002400
00002478
00002FDA
000030C7
0000352A
000035D9
00003691
000037D3
00003848
000038BE
00003E5B
00003FAD
000040B3
00004468
00004510
00004943
00004E2A
00005018
0000565C
00005DD2
000063C4
00006431
000074E5
0000763C
00007CE3
00007E13
0000811E
000084C8
0000868E
00008891
00008B22
00008C1B
00008E89
00008EA0
000090BE
000090DC
0000991B
00009D8F
0000AA56
0000AF0D
0000B5CD
0000C2A9
0000C65E
0000C788
0000CAAB
0000CC09
0000CFCD
0000DAD2
0000DEB2
0000E5FA
0000E6FC
0000E792
0000E8EB
0000EB2F
0000EF8B
0000EFFD
0000F262
0000F33F
0000F612
0000F633
0000F663
0000F810
0000F8F8
0000F955
0000FA9A
0000FEFA


In [ ]:
# defining list of missing participants 
# (only 12 missing participants here, rest were distributed evenly in 6 seperate batches)
missing_patients = ["00000172", "000003B4", "00000A67", "00000EAE", "00001573", 
                    "00002400", "00002478", "00002FDA", "000030C7", "0000352A", 
                    "000035D9", "00003691"]


patients_data = {}

# first batch of missing participants
output_file = 'mri_data_LC_missing_patients.csv'

header = ['Patient Number']
all_keys = set()

# saving current results to csv file
def save_progress():
    output_data = []
    for patient_number, intensities in patients_data.items():
        row = [patient_number]
        for col in header[1:]:
            row.append(intensities.get(col, None))  # Use None for missing values
        output_data.append(row)

    output_df = pd.DataFrame(output_data, columns=header)
    output_df.to_csv(output_file, index=False)
    print(f"Progress saved to {output_file}")

patient_count = 0

try:
    # loop through subdirectories 'FSL' in base_dir (recursively)
    for root, dirs, files in os.walk(base_dir):
        if "FSL" in dirs:
            fsl_dir = os.path.join(root, "FSL")
            
            # extract participant number from folder name 
            patient_number = os.path.basename(root)

            # process only the specified missing participants
            if patient_number not in missing_patients:
                continue

            # loop through NIfTI files in the FSL directory
            for filename in os.listdir(fsl_dir):
                # process only files with "mni", "nii.gz"
                if (
                    "mni" in filename
                    and "mask" not in filename
                    and "new" not in filename
                    and "flirt" not in filename
                    and "flirt_masked" not in filename
                    and not filename.endswith(".mat")
                    and filename.endswith(".nii.gz")
                ):
                    input_file = os.path.join(fsl_dir, filename)

                    
                    print(f"Processing file: {input_file}")

                    # loop through the LC masks (left, right, combined) - each MRI file was processed three times
                    for mask_file in [LC_mask, mask_lc_r, mask_lc_l]:
                        try:
                            # use FLIRT to perform the registration of the LC masks
                            flirt_output = f"{input_file}_flirt_{os.path.basename(mask_file).split('.')[0]}"
                            flirt_matrix = f"{flirt_output}.mat"
                            flirt_command = f"flirt -in {input_file} -ref {mask_file} -out {flirt_output} -omat {flirt_matrix}"
                            process_flirt = subprocess.run(flirt_command, shell=True, capture_output=True, text=True)

                            if process_flirt.returncode != 0:
                                print(f"Error applying FLIRT to {filename} with mask {mask_file}: {process_flirt.stderr}")
                                continue  # Continue with the next mask for same file

                            # use fslmaths to apply the masks and threshold negative values to zero
                            flirt_masked = f"{flirt_output}_masked"
                            fslmaths_command = f"fslmaths {flirt_output} -mas {mask_file} -thr 0 {flirt_masked}"
                            process_fslmaths = subprocess.run(fslmaths_command, shell=True, capture_output=True, text=True)

                            if process_fslmaths.returncode != 0:
                                print(f"Error applying fslmaths to {filename} with mask {mask_file}: {process_fslmaths.stderr}")
                                continue  # Continue with the next mask for same file

                            # use fslstats to extract mean intensity within the region
                            stats_command = f"fslstats {flirt_masked} -M"  # -M for Mean Intensity 
                            process_stats = subprocess.run(stats_command, shell=True, capture_output=True, text=True)

                            if process_stats.returncode != 0:
                                print(f"Error processing {filename}: {process_stats.stderr}")
                                continue  # continue with the next mask for same file

                            
                            stats = process_stats.stdout.strip().split()
                            mean_intensity = float(stats[0])  

                            # remove intermediate files and temporary NifTI outputs
                            try:
                                for file_to_delete in [flirt_matrix, flirt_output + ".nii.gz", flirt_masked + ".nii.gz"]:
                                    if os.path.exists(file_to_delete):
                                        os.remove(file_to_delete)
                                    else:
                                        print(f"File not found: {file_to_delete}")
                            except OSError as e:
                                print(f"Error deleting files for {filename}: {e}")

                            # add data to the participant's entry
                            if patient_number not in patients_data:
                                patients_data[patient_number] = {}

                            mask_name = os.path.basename(mask_file).split('.')[0]  
                            filename_base = filename.replace('.nii.gz', '') 
                            mask_key = f'{mask_name}_{filename_base}'

                            patients_data[patient_number][mask_key] = mean_intensity

                        except Exception as e:
                            print(f"Error processing {filename} with mask {mask_file} for patient {patient_number}: {e}")
                            continue  # continue to next mask or next file

                    patient_count += 1
                    print(f"Finished processing patient {patient_number}, saving progress...")

                    # saving mean intensity immediately after each file is processed with all three masks
                    save_progress()

    # final save after all participants and files have been processed
    print("Final saving progress...")
    save_progress()

# handle interruptions and unexpected errors
except KeyboardInterrupt:
    print("Process interrupted, saving progress...")
    save_progress()

except Exception as e:
    print(f"An error occurred: {e}")
    save_progress()

Processing file: /hus/home/oliwin/data/study_name/000003B4/FSL/mprage_reg_mni.nii.gz
Finished processing patient 000003B4, saving progress...
Progress saved to mri_data_LC_missing_patients.csv
Processing file: /hus/home/oliwin/data/study_name/000003B4/FSL/STAGE_CROWN_PD_MAP_reg_mni.nii.gz
Finished processing patient 000003B4, saving progress...
Progress saved to mri_data_LC_missing_patients.csv
Processing file: /hus/home/oliwin/data/study_name/000003B4/FSL/STAGE_CROWN_R2S_reg_mni.nii.gz
Finished processing patient 000003B4, saving progress...
Progress saved to mri_data_LC_missing_patients.csv
Processing file: /hus/home/oliwin/data/study_name/000003B4/FSL/STAGE_CROWN_R2S_A2_reg_mni.nii.gz
Finished processing patient 000003B4, saving progress...
Progress saved to mri_data_LC_missing_patients.csv
Processing file: /hus/home/oliwin/data/study_name/000003B4/FSL/STAGE_CROWN_T2S_reg_mni.nii.gz
Finished processing patient 000003B4, saving progress...
Progress saved to mri_data_LC_missing_patien

In [ ]:

print(patients_data)

{'000003B4': {'LC_mask_mprage_reg_mni': 2.0, 'LC_R_mask_mprage_reg_mni': 2.0, 'LC_L_mask_mprage_reg_mni': 2.0, 'LC_mask_STAGE_CROWN_PD_MAP_reg_mni': 606.30922, 'LC_R_mask_STAGE_CROWN_PD_MAP_reg_mni': 598.081633, 'LC_L_mask_STAGE_CROWN_PD_MAP_reg_mni': 558.646409, 'LC_mask_STAGE_CROWN_R2S_reg_mni': 3066.191489, 'LC_R_mask_STAGE_CROWN_R2S_reg_mni': 3015.489796, 'LC_L_mask_STAGE_CROWN_R2S_reg_mni': 2774.718232, 'LC_mask_STAGE_CROWN_R2S_A2_reg_mni': 3696.085106, 'LC_R_mask_STAGE_CROWN_R2S_A2_reg_mni': 4795.402332, 'LC_L_mask_STAGE_CROWN_R2S_A2_reg_mni': 2555.723757, 'LC_mask_STAGE_CROWN_T2S_reg_mni': 4704.710638, 'LC_R_mask_STAGE_CROWN_T2S_reg_mni': 4783.64723, 'LC_L_mask_STAGE_CROWN_T2S_reg_mni': 4493.541436, 'LC_mask_STAGE_CROWN_T2S_A2_reg_mni': 3687.293617, 'LC_R_mask_STAGE_CROWN_T2S_A2_reg_mni': 3683.279883, 'LC_L_mask_STAGE_CROWN_T2S_A2_reg_mni': 3947.464088, 'LC_mask_STAGE_CROWN_TRUE_PD_MAP_reg_mni': 656.541844, 'LC_R_mask_STAGE_CROWN_TRUE_PD_MAP_reg_mni': 646.189504, 'LC_L_mask_STAG

In [ ]:
# preparing dataframe to store mean intensities
header = ['Patient Number']
all_keys = set()
for intensities in patients_data.values():
    all_keys.update(intensities.keys())
header.extend(sorted(all_keys))  

output_data = []
for patient_number, intensities in patients_data.items():
    row = [patient_number]
    for col in header[1:]:
        row.append(intensities.get(col, None))  # using None for missing values
    output_data.append(row)

# creating dataframe
output_df = pd.DataFrame(output_data, columns=header)

# saving dataframe as csv for further analysis
output_file = 'mri_data_LC_missing_patients.csv' 
output_df.to_csv(output_file, index=False)
print(f"Transformed CSV saved to {output_file}")

Transformed CSV saved to mri_data_LC_missing_patients.csv


In [ ]:
# 7th list of missing participants
# this list includes 10 of the missing participants from first extraction
missing_patients = [
    "00000172", "00000A67", "00000EAE", "00001573", 
    "00002400", "00002478", "000030C7", "0000352A", 
    "000035D9", "00003691"
]

patients_data = {}

# defining missing list nr 7
output_file = 'mri_data_LC_missing_patients_7.csv'


header = ['Patient Number']
all_keys = set()

#saving current results to csv file
def save_progress():
    global header

  
    all_keys.update(*(intensities.keys() for intensities in patients_data.values()))
    header = ['Patient Number'] + sorted(all_keys)

    output_data = []
    for patient_number, intensities in patients_data.items():
        row = [patient_number]
        for col in header[1:]:
            row.append(intensities.get(col, None))  # use None for missing values
        output_data.append(row)

    output_df = pd.DataFrame(output_data, columns=header)
    output_df.to_csv(output_file, index=False)
    print(f"Progress saved to {output_file}")


try:
    # loop through subdirectories 'FSL' in base_dir (recursively)
    for root, dirs, files in os.walk(base_dir):
        if "FSL" in dirs:
            fsl_dir = os.path.join(root, "FSL")
            
            # extracting participant number from folder name 
            patient_number = os.path.basename(root)

            # processing only specified missing participants
            if patient_number not in missing_patients:
                continue

            # loop through NIfTI files in FSL directory
            for filename in os.listdir(fsl_dir):
                # process only files with "mni", "nii.gz"
                if (
                    "mni" in filename
                    and "mask" not in filename
                    and "new" not in filename
                    and "flirt" not in filename
                    and "flirt_masked" not in filename
                    and not filename.endswith(".mat")
                    and filename.endswith(".nii.gz")
                ):
                    input_file = os.path.join(fsl_dir, filename)

                    
                    print(f"Processing file: {input_file}")

                    # loop through LC masks (left, right, combined) - each MRI file is processed three times
                    for mask_file in [LC_mask, mask_lc_r, mask_lc_l]:
                        try:
                            # use FLIRT to perform the registration of the LC masks
                            flirt_output = f"{input_file}_flirt_{os.path.basename(mask_file).split('.')[0]}"
                            flirt_matrix = f"{flirt_output}.mat"
                            flirt_command = f"flirt -in {input_file} -ref {mask_file} -out {flirt_output} -omat {flirt_matrix}"
                            process_flirt = subprocess.run(flirt_command, shell=True, capture_output=True, text=True)

                            if process_flirt.returncode != 0:
                                print(f"Error applying FLIRT to {filename} with mask {mask_file}: {process_flirt.stderr}")
                                continue  # continue with the next mask for same file

                            # use fslmaths to apply the masks and threshold negative values to zero
                            flirt_masked = f"{flirt_output}_masked"
                            fslmaths_command = f"fslmaths {flirt_output} -mas {mask_file} -thr 0 {flirt_masked}"
                            process_fslmaths = subprocess.run(fslmaths_command, shell=True, capture_output=True, text=True)

                            if process_fslmaths.returncode != 0:
                                print(f"Error applying fslmaths to {filename} with mask {mask_file}: {process_fslmaths.stderr}")
                                continue  # continue with the next mask for same file

                            # use fslstats to extract mean intensity within the region
                            stats_command = f"fslstats {flirt_masked} -M"  # -M for Mean Intensity 
                            process_stats = subprocess.run(stats_command, shell=True, capture_output=True, text=True)

                            if process_stats.returncode != 0:
                                print(f"Error processing {filename}: {process_stats.stderr}")
                                continue  # continue with the next mask for same file

                           
                            stats = process_stats.stdout.strip().split()
                            mean_intensity = float(stats[0])  

                            # remove intermediate files and temporary NIfTI outputs
                            try:
                                for file_to_delete in [flirt_matrix, flirt_output + ".nii.gz", flirt_masked + ".nii.gz"]:
                                    if os.path.exists(file_to_delete):
                                        os.remove(file_to_delete)
                                    else:
                                        print(f"File not found: {file_to_delete}")
                            except OSError as e:
                                print(f"Error deleting files for {filename}: {e}")

                            # add the data to the participant's entry
                            if patient_number not in patients_data:
                                patients_data[patient_number] = {}

                            mask_name = os.path.basename(mask_file).split('.')[0]  
                            filename_base = filename.replace('.nii.gz', '') 
                            mask_key = f'{mask_name}_{filename_base}'

                            patients_data[patient_number][mask_key] = mean_intensity

                            # saving mean intensity immediately after each file is processed with all three masks
                            save_progress()

                        except Exception as e:
                            print(f"Error processing {filename} with mask {mask_file} for patient {patient_number}: {e}")
                            continue  # continue to the next mask or next file

    # final save after all participants and files have been processed                      
    print("Final saving progress...")
    save_progress()

# handle interruptions and unexpected errors
except KeyboardInterrupt:
    print("Process interrupted, saving progress...")
    save_progress()

except Exception as e:
    print(f"An error occurred: {e}")
    save_progress()

Processing file: /hus/home/oliwin/data/study_name/00000A67/FSL/mprage_reg_mni.nii.gz
Progress saved to mri_data_LC_missing_patients_7.csv
Progress saved to mri_data_LC_missing_patients_7.csv
Progress saved to mri_data_LC_missing_patients_7.csv
Processing file: /hus/home/oliwin/data/study_name/00000A67/FSL/STAGE_CROWN_PD_MAP.nii.gz_reg_mni.nii.gz
Progress saved to mri_data_LC_missing_patients_7.csv
Progress saved to mri_data_LC_missing_patients_7.csv
Progress saved to mri_data_LC_missing_patients_7.csv
Processing file: /hus/home/oliwin/data/study_name/00000A67/FSL/STAGE_CROWN_R2S.nii.gz_reg_mni.nii.gz
Progress saved to mri_data_LC_missing_patients_7.csv
Progress saved to mri_data_LC_missing_patients_7.csv
Progress saved to mri_data_LC_missing_patients_7.csv
Processing file: /hus/home/oliwin/data/study_name/00000A67/FSL/STAGE_CROWN_R2S_A2.nii.gz_reg_mni.nii.gz
Progress saved to mri_data_LC_missing_patients_7.csv
Progress saved to mri_data_LC_missing_patients_7.csv
Progress saved to mri_d